In [1]:
import json
import numpy as np
import pandas as pd
import time
import math
import numpy

import sys
sys.path.append('../GillesPy2/')
import gillespy2
from gillespy2 import Model, Species, Parameter, Reaction, Event, \
                      EventTrigger, EventAssignment, export_StochSS
from gillespy2 import ODECSolver, ODESolver, SSACSolver

import matplotlib.pyplot as plt

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
import ipywidgets as widgets
from IPython.display import display

from scipy import spatial

# Devils DFTD Parameter Approximation

This notebook is used to calibrate a initial values which approximately match the results given in Cunningham, Calum X., et al. "Quantifying 25 years of disease‐caused declines in Tasmanian devil populations: host density drives spatial pathogen spread." Ecology Letters 24.5 (2021): 958-969. The objective of this notebook is to gain an "eyeball approximation" of reasonable starting values to serve as input for more sophisticated inferencing.

## Read in observed data

In [2]:
# pop_data = pd.read_csv('../year_data/devil_data_v2.csv')
pop_data = pd.read_csv('month_data/Devils_Dataset__Population_1985-2020.csv')

devil_pop = np.array(pop_data['Population'].iloc[:].values)
dates = []
year = 1985
while len(dates) < len(devil_pop):
    for month in ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sept", "Oct", "Nov", "Dec"]:
        dates.append(f"{month} {year}")
    year += 1

obs = np.vstack([devil_pop]).reshape(1, 1, -1)
carry_cap = int(max(devil_pop)*1.16)

In [3]:
plt.figure(figsize=[15, 8])
plt.title("Tasmanian Devil Population with DFTD: Observed", fontsize=18)
plt.xlabel(f"Time (months) since {dates[0]}", fontsize=14)
plt.ylabel("Population of Tasmanian Devils", fontsize=14)
plt.plot([132, 132], [-3000, max(devil_pop)*1.16], '--r')
plt.ylim([-3000, max(devil_pop)*1.16])
plt.text(125, 15000, "DFTD First Observed", rotation="vertical", color="red", fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12, rotation="vertical", va="center")
_ = plt.plot(range(len(devil_pop)), devil_pop, 'b', label='Observed Total')

## Model

In [4]:
variables = {
    "juvenile_concentration": 0.467,
    "infected_concentration": 5e-5,
    "birth_rate": 0.055,
    "maturity_rate": 0.04267,
    "infection_rate_infected": 1.675e-05,
    "infection_rate_diseased": 4e-05,
    "incubation": 19,
    "progression": 12,
    "death_rate_juvenile": 0.006,
    "death_rate_susceptible": 0.02335,
    "death_rate_over_population": 2.3e-7,
    "death_rate_infected": 0.03,
    "death_rate_diseased": 0.024,
    "DFTD_start": 95
}
variables_orig = variables.copy()

In [5]:
class DevilsPreVaccination(Model):
    def __init__(self, values=variables, events=None):
        Model.__init__(self, name="Devils DFTD 2-Stage Infection")
        self.volume = 1

        # Parameters
        birth_rate = Parameter(name="birth_rate", expression=values['birth_rate'])
        death_rate_juvenile = Parameter(name="death_rate_juvenile", expression=values['death_rate_juvenile'])
        maturity_rate = Parameter(name="maturity_rate", expression=values['maturity_rate'])
        death_rate_susceptible = Parameter(
            name="death_rate_susceptible", expression=values['death_rate_susceptible']
        )
        death_rate_over_population = Parameter(
            name="death_rate_over_population", expression=values['death_rate_over_population']
        )
        infection_rate_infected = Parameter(
            name="infection_rate_infected", expression=values['infection_rate_infected']
        )
        infection_rate_diseased = Parameter(
            name="infection_rate_diseased", expression=values['infection_rate_diseased']
        )
        incubation = Parameter(name="incubation", expression=values['incubation'])
        death_rate_infected = Parameter(name="death_rate_infected", expression=values['death_rate_infected'])
        progression = Parameter(name="progression", expression=values['progression'])
        death_rate_diseased = Parameter(name="death_rate_diseased", expression=values['death_rate_diseased'])
        DFTD_start = Parameter(name="DFTD_start", expression=values['DFTD_start'])
        self.add_parameter([birth_rate, death_rate_juvenile, maturity_rate, death_rate_susceptible,
                            death_rate_over_population, infection_rate_infected, infection_rate_diseased,
                            incubation, death_rate_infected, progression, death_rate_diseased, DFTD_start])

        # Variables (initial values adjusted to observed data)
        initial_devil_population  = int(devil_pop[0])
        
        Juvenile = Species(
            name="Juvenile", mode="discrete",
            initial_value=round(initial_devil_population * values['juvenile_concentration'])
        )
        Susceptible = Species(
            name="Susceptible", mode="discrete",
            initial_value=round(initial_devil_population * (1 - values['juvenile_concentration']))
        )
        Exposed = Species(name="Exposed", initial_value=0, mode="discrete")
        Infected = Species(name="Infected", initial_value=0, mode="discrete")
        Diseased = Species(name="Diseased", initial_value=0, mode="discrete")
        Devils = Species(name="Devils", initial_value=initial_devil_population, mode="discrete")
        self.add_species([Juvenile, Susceptible, Exposed, Infected, Diseased, Devils])

        # Reactions
        Birth = Reaction(name="Birth",
            reactants={}, products={'Juvenile': 1, 'Devils': 1},
            propensity_function="birth_rate * (Susceptible + Exposed + Infected)"
        )
        Mature = Reaction(name="Mature",
            reactants={'Juvenile': 1}, products={'Susceptible': 1},
            propensity_function="Juvenile * maturity_rate"
        )
        self.add_reaction([Birth, Mature])
        
        Death_Juvenile = Reaction(name="Death_Juvenile",
            reactants={'Juvenile': 1, 'Devils': 1}, products={},
            propensity_function="death_rate_juvenile * Juvenile"
        )
        Death_Juvenile2 = Reaction(name="Death_Juvenile2",
            reactants={'Juvenile': 1, 'Devils': 1}, products={},
            propensity_function="death_rate_over_population * Juvenile * (Devils-1)"
        )
        Death_Susceptible = Reaction(name="Death_Susceptible",
            reactants={'Susceptible': 1, 'Devils': 1}, products={},
            propensity_function="death_rate_susceptible * Susceptible"
        )
        Death_Susceptible2 = Reaction(name="Death_Susceptible2",
            reactants={'Susceptible': 1, 'Devils': 1}, products={},
            propensity_function="death_rate_over_population * Susceptible * (Devils-1)"
        )
        Death_Infected = Reaction(name="Death_Infected",
            reactants={'Infected': 1, 'Devils': 1}, products={},
            propensity_function="death_rate_infected * Infected"
        )
        Death_Infected2 = Reaction(name="Death_Infected2",
            reactants={'Infected': 1, 'Devils': 1}, products={},
            propensity_function="death_rate_over_population * Infected * (Devils-1)"
        )
        Death_Exposed = Reaction(name="Death_Exposed",
            reactants={'Devils': 1, 'Exposed': 1}, products={},
            propensity_function="death_rate_susceptible * Exposed"
        )
        Death_Exposed2 = Reaction(name="Death_Exposed2",
            reactants={'Devils': 1, 'Exposed': 1}, products={},
            propensity_function="death_rate_over_population * Exposed * (Devils - 1)"
        )
        Death_Diseased = Reaction(name="Death_Diseased",
            reactants={'Diseased': 1, 'Devils': 1}, products={},
            propensity_function="death_rate_diseased * Diseased"
        )
        Death_Diseased2 = Reaction(name="Death_Diseased2",
            reactants={'Diseased': 1, 'Devils': 1}, products={},
            propensity_function="death_rate_over_population * Diseased * (Devils - 1)"
        )
        self.add_reaction([Death_Juvenile, Death_Susceptible, Death_Exposed, Death_Infected, Death_Diseased,
                           Death_Juvenile2, Death_Susceptible2, Death_Exposed2, Death_Infected2, Death_Diseased2])
        
        
        TransmissionI = Reaction(name="TransmissionI",
            reactants={'Susceptible': 1, 'Infected': 1}, products={'Exposed': 1, 'Infected':1},
            propensity_function="infection_rate_infected * Susceptible * Infected"
        )
        TransmissionD = Reaction(name="TransmissionD",
            reactants={'Susceptible': 1, 'Diseased': 1}, products={'Exposed': 1, 'Diseased':1},
            propensity_function="infection_rate_diseased * Susceptible * Diseased"
        )
        DFTD_Stage1 = Reaction(name="DFTD_Stage1",
            reactants={'Exposed': 1}, products={'Infected': 1},
            propensity_function="Exposed / incubation"
        )
        DFTD_Stage2 = Reaction(name="DFTD_Stage2",
            reactants={'Infected': 1}, products={'Diseased': 1},
            propensity_function="Infected / progression"
        )
        self.add_reaction([TransmissionI, TransmissionD, DFTD_Stage1, DFTD_Stage2])

        # Events
        if events is not None:
            self.add_event(events)

        # Timespan
        self.timespan(np.arange(0, 421, 1)) # month data tspan
        


In [6]:
def event_model(incubation=None):
    if incubation is None:
        incubation = variables['incubation']
    carry_cap = int(max(devil_pop)*1.16)
    DFTD_Introduction = Event(name="DFTD_Introduction", assignments= [
        EventAssignment(variable="Infected", expression="1"),
        EventAssignment(variable="Susceptible", expression="Susceptible - 1")
    ], trigger=EventTrigger(expression=f"t >= {variables['DFTD_start']}"))
    var2 = variables.copy()
    var2['incubation']=incubation
    e_model = DevilsPreVaccination(values=var2, events=[DFTD_Introduction])
    return e_model


In [7]:
from gillespy2 import TauHybridCSolver

In [8]:
emodel = event_model()
tic = time.time()
eresults = emodel.run(number_of_trajectories=100, solver=TauHybridCSolver)
print(f"simulation complete in {time.time()-tic}s")

simulation complete in 60.818928718566895s


In [9]:
len(eresults)

100

<ipython-input-10-6769b8e51795>:1: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition,mark_inset)
<ipython-input-10-6769b8e51795>:66: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [10]:
from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition,mark_inset)
fig, ax1 = plt.subplots(figsize=[12, 6])

plt.title("Tasmanian Devil Population with DFTD: Observed vs. Simulated", fontsize=18)
ax1.set_xlabel(f"Time (months) since {dates[0]}", fontsize=16)
ax1.set_ylabel("Population of Tasmanian Devils", fontsize=16)
ax1.plot(eresults[0]['time'], obs[0][0], '--', color='black', label='Observed Total')
ax1.plot(eresults[0]['time'], eresults[0]['Devils'], color='blue', label='Simulated Total')
ax1.plot(eresults[0]['time'], eresults[0]['Juvenile'], color='purple', alpha=.6, label='Juvenile')
ax1.plot(eresults[0]['time'], eresults[0]['Susceptible'], color='green', alpha=.6, label='Susceptible')
ax1.plot(eresults[0]['time'], eresults[0]['Exposed'], color='magenta', alpha=.6, label='Exposed')
ax1.plot(eresults[0]['time'], eresults[0]['Infected'], color='red', alpha=.6, label='Infected')
ax1.plot(eresults[0]['time'], eresults[0]['Diseased'], color='cyan', alpha=.6, label='Diseased')
ax1.plot([variables['DFTD_start'], variables['DFTD_start']], [-3000, carry_cap], '--k', alpha=0.4)
ax1.text(variables['DFTD_start']-7, 5000, "DFTD Start", rotation="vertical", color="black", fontsize=14)
ax1.text(variables['DFTD_start']-7, 24000, dates[variables['DFTD_start']], rotation="vertical", color="black", fontsize=14)
#ax1.set_xlim([0, result[0]['time'][-1]])
#ax1.set_ylim([-3000, max(devil_pop)*1.25])
ax1.tick_params(axis='x', labelsize=12)
ax1.set_yticks([20000,40000,60000])
ax1.tick_params(axis='y',labelsize=12, labelrotation=90)
ax1.legend(loc='upper right', fontsize=16)
ax1.set_ylim([0, carry_cap])
ax1.set_xlim(0,eresults[0]['time'][-1])
for n,r in enumerate(eresults):
    if n==0: continue
    ax1.plot(eresults[n]['time'], eresults[n]['Devils'],'b', alpha=0.025)
    ax1.plot(eresults[n]['time'], eresults[n]['Juvenile'], color='purple', alpha=0.025, label='Juvenile')
    ax1.plot(eresults[n]['time'], eresults[n]['Susceptible'], color='green', alpha=0.025, label='Susceptible')
    ax1.plot(eresults[n]['time'], eresults[n]['Exposed'], color='magenta', alpha=0.025, label='Exposed')
    ax1.plot(eresults[n]['time'], eresults[n]['Infected'], color='red', alpha=0.025, label='Infected')
    ax1.plot(eresults[n]['time'], eresults[n]['Diseased'], color='cyan', alpha=0.025, label='Diseased')

ax2 = plt.axes([0,0,1,1])
# Manually set the position and relative size of the inset axes within ax1
ip = InsetPosition(ax1, [0.275,0.5,0.175,0.25])
ax2.set_axes_locator(ip)
# Mark the region corresponding to the inset axes on ax1 and draw lines
# in grey linking the two axes.
mark_inset(ax1, ax2, loc1=4, loc2=3, fc="none", ec='0.5')

# plot in the inset
inst=135
ined=inst+31
#ax2.plot(range(len(devil_pop))[inst:ined], devil_pop[inst:ined], '--k', label='Observed Total')
#ax2.plot(eresults[0]['time'][inst:ined], eresults[0]['Devils'][inst:ined],'b', label="Simulation")
ax2.plot(eresults[0]['time'], eresults[0]['Juvenile'], color='purple', alpha=.6, label='Juvenile')
ax2.plot(eresults[0]['time'], eresults[0]['Susceptible'], color='green', alpha=.6, label='Susceptible')
ax2.plot(eresults[0]['time'], eresults[0]['Exposed'], color='magenta', alpha=.6, label='Exposed')
ax2.plot(eresults[0]['time'], eresults[0]['Infected'], color='red', alpha=.6, label='Infected')
ax2.plot(eresults[0]['time'], eresults[0]['Diseased'], color='cyan', alpha=.6, label='Diseased')
for n,r in enumerate(eresults):
    if n==0: continue
    #ax2.plot(eresults[n]['time'][inst:ined], eresults[n]['Devils'][inst:ined],'b', alpha=0.05)
    ax2.plot(eresults[n]['time'], eresults[n]['Devils'],'b', alpha=0.03)
    ax2.plot(eresults[n]['time'], eresults[n]['Juvenile'], color='purple', alpha=0.025, label='Juvenile')
    ax2.plot(eresults[n]['time'], eresults[n]['Susceptible'], color='green', alpha=0.025, label='Susceptible')
    ax2.plot(eresults[n]['time'], eresults[n]['Exposed'], color='magenta', alpha=0.025, label='Exposed')
    ax2.plot(eresults[n]['time'], eresults[n]['Infected'], color='red', alpha=0.025, label='Infected')
    ax2.plot(eresults[n]['time'], eresults[n]['Diseased'], color='cyan', alpha=0.025, label='Diseased')
ax2.set_xlim(inst,ined-1)
ax2.set_ylim(2000,7000)
#ax2.set_yticks([49000,54000])
ax2.tick_params(axis='y', labelrotation=90)
    
plt.tight_layout()
#plt.savefig("DevilsDFTD_Data_DFTDsim.pdf")

In [11]:
import numpy

In [12]:
ps_incubation_db={}

In [13]:
def sample_start_time(incubation):
    global obs
    total_results = 100
    num_results = 0
    dists = numpy.zeros(total_results)
    while(num_results < total_results):
        ntraj = total_results-num_results
        print(f"\tRunning {ntraj}", end=' ')
        sys.stdout.flush()
        tic = time.time()
        eresults = event_model(incubation=incubation).run(number_of_trajectories=ntraj, solver=TauHybridCSolver)
        print(f"\tsimulation complete in {time.time()-tic}s")
        for r in eresults:
            if (r['Exposed'][-1] + r['Infected'][-1] + r['Diseased'][-1]) > 0:
                dists[num_results] = numpy.linalg.norm(r['Devils']-obs[0][0],2)
                num_results+=1
    d = numpy.average(dists)
    print(f"incubation={incubation} L2={d}")
    return d

In [14]:
ps_incubation_sweep=[15,16,17,18,19,20,21,22,23,24,25]

In [15]:
for incubation in ps_incubation_sweep:
    if incubation in ps_incubation_db:
        print(f"incubation={incubation} already calculated L2={ps_incubation_db[incubation]}")
    else:
        print(f"Running incubation={incubation}")
        ps_incubation_db[incubation] = sample_start_time(incubation)

Running incubation=15
	Running 100 	simulation complete in 64.20744776725769s
incubation=15 L2=79491.14401833522
Running incubation=16
	Running 100 	simulation complete in 73.53469824790955s
	Running 14 	simulation complete in 26.65290641784668s
	Running 2 	simulation complete in 22.760674238204956s
incubation=16 L2=64627.6428415541
Running incubation=17
	Running 100 	simulation complete in 82.35791015625s
incubation=17 L2=61540.0535377877
Running incubation=18
	Running 100 	simulation complete in 37.474422216415405s
	Running 100 	simulation complete in 92.67698550224304s
incubation=18 L2=47427.90134258559
Running incubation=19
	Running 100 	simulation complete in 78.189462184906s
incubation=19 L2=39146.56839524331
Running incubation=20
	Running 100 	simulation complete in 88.92638158798218s
incubation=20 L2=37071.523893131736
Running incubation=21
	Running 100 	simulation complete in 27.646095275878906s
	Running 100 	simulation complete in 72.66648268699646s
incubation=21 L2=37929.008

In [16]:
ps_incubation_dist=[]
for incubation in ps_incubation_sweep:
    ps_incubation_dist.append(ps_incubation_db[incubation])

plt.figure(figsize=[12, 6])
plt.title("Simulation Error vs. incubation", fontsize=18)
    
plt.plot(ps_incubation_sweep,ps_incubation_dist,'.-')
plt.xlabel('incubation')
plt.ylabel("E[L2(data-sim)]")
plt.yticks([60000,80000,100000,120000],fontsize=12, rotation="vertical", va="center")
plt.xlim(ps_incubation_sweep[0],ps_incubation_sweep[-1])
plt.tight_layout()

In [17]:
ps_incubation_db

{15: 79491.14401833522,
 16: 64627.6428415541,
 17: 61540.0535377877,
 18: 47427.90134258559,
 19: 39146.56839524331,
 20: 37071.523893131736,
 21: 37929.00872669573,
 22: 38282.55345208549,
 23: 59575.424123154095,
 24: 77957.43216584327,
 25: 73317.38665215114}

In [18]:
maxval=math.inf
maxkey=''
for k,v in ps_incubation_db.items():
    if v<maxval:
        maxkey=k
        maxval=v
print(f"Selected value {maxkey}")

Selected value 20
